In [1]:
from main import *

In [2]:
t = pd.read_csv('processed_data/msci_us_sri/historical.csv')

In [3]:
t['ticker'].nunique()

192

In [4]:
t.value_counts('date').sort_index().plot()

In [5]:
t['date'] = pd.to_datetime(t['date']).dt.date

In [21]:
t['exists'] = 1

In [39]:
tl = pd.merge(
	pd.DataFrame({'date': t['date'].unique()}),
	pd.DataFrame({'ticker': t['ticker'].unique()}),
	how='cross'
)
tl = pd.merge(tl, t, on=['date', 'ticker'], how='left')
tl['exists'] = tl['exists'].fillna(0)
tl = tl.sort_values(['date', 'ticker']).reset_index(drop=True)

# add columns for whether a ticker was added or dropped on a given day
tl['added_today'] = (tl['exists'] == 1) & (tl.groupby('ticker')['exists'].shift(1) == 0)
tl['dropped_today'] = (tl['exists'] == 0) & (tl.groupby('ticker')['exists'].shift(1) == 1)

In [ ]:
tl['added_today'] = (tl['exists'] == 1) & (tl.groupby('ticker')['exists'].shift(1) == 0)
tl['dropped_today'] = (tl['exists'] == 0) & (tl.groupby('ticker')['exists'].shift(1) == 1)

,date,ticker,rank,name,sector,weight,exists,added_today,dropped_today
0,2025-04-01,A,86.00,AGILENT TECHNOLOGIES INC,Health Care,0.27,1.00,False,False
1,2025-04-01,ADBE,9.00,ADOBE INC,Information Technology,1.82,1.00,False,False
2,2025-04-01,ADP,24.00,AUTOMATIC DATA PROCESSING INC,Industrials,1.03,1.00,False,False
3,2025-04-01,ADSK,37.00,AUTODESK INC,Information Technology,0.62,1.00,False,False
4,2025-04-01,ALLE,158.00,ALLEGION PLC,Industrials,0.09,1.00,False,False
...,...,...,...,...,...,...,...,...,...
12475,2025-06-30,WTRG,NaN,NaN,NaN,NaN,0.00,False,False
12476,2025-06-30,WTW,89.00,WILLIS TOWERS WATSON PLC,Financials,0.24,1.00,False,False
12477,2025-06-30,XYL,86.00,XYLEM INC,Industrials,0.24,1.00,False,False
12478,2025-06-30,ZBH,126.00,ZIMMER BIOMET HOLDINGS INC,Health Care,0.14,1.00,False,False


In [43]:
qt.view(tl[tl['date'] == dt.date(2025, 5, 30)])

Grid(columns_fit='auto', compress_data=True, css_rules_down=['.number-cell {text-align: left;width: 10;}', '.l…

In [45]:
tl_rebal = tl[tl['date'] == dt.date(2025, 5, 30)]
tl_rebal[tl_rebal['dropped_today']].shape
tl_rebal[tl_rebal['added_today']].shape

(28, 9)

(21, 9)

In [46]:
tl_rebal

,date,ticker,rank,name,sector,weight,exists,added_today,dropped_today
8256,2025-05-30,A,85.00,AGILENT TECHNOLOGIES INC,Health Care,0.26,1.00,False,False
8257,2025-05-30,ADBE,7.00,ADOBE INC,Information Technology,2.93,1.00,False,False
8258,2025-05-30,ADP,22.00,AUTOMATIC DATA PROCESSING INC,Industrials,1.07,1.00,False,False
8259,2025-05-30,ADSK,23.00,AUTODESK INC,Information Technology,1.05,1.00,False,False
8260,2025-05-30,ALLE,146.00,ALLEGION PLC,Industrials,0.10,1.00,False,False
...,...,...,...,...,...,...,...,...,...
8443,2025-05-30,WTRG,NaN,NaN,NaN,NaN,0.00,False,True
8444,2025-05-30,WTW,87.00,WILLIS TOWERS WATSON PLC,Financials,0.25,1.00,True,False
8445,2025-05-30,XYL,88.00,XYLEM INC,Industrials,0.25,1.00,False,False
8446,2025-05-30,ZBH,125.00,ZIMMER BIOMET HOLDINGS INC,Health Care,0.15,1.00,False,False
